  # - Object Detection and Counting -

                                                 Realized by : Maleke KOUKI       

    This file displays the testing of object detection and counting on images, videos and our camera lens, which is our primary object. 
    
    During this project, we decided to employ the SSD method, which is based on the combination of YOLO for its accuracy and R-CNN for its velocity. For performance considerations, we'll use MobileNetV2. 
    SSD (Single Shot MultiBox Detector) is an object detection architecture based on a CNN architecture, while MobileNetV2 is a CNN architecture optimized for feature extraction. They are often used together in object detection models to take advantage of MobileNetV2's efficiency and SSD's object detection capabilities, creating a balanced and efficient object detection system.


# Import libraries

- Capturer des flux vidéo depuis la caméra et l'affichage du résultat

In [1]:
pip install opencv-python 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2

# Get OpenCV version
print("OpenCV version:", cv2.__version__)

# Check if OpenCV is available
if cv2:
    print("OpenCV is properly installed.")
else:
    print("OpenCV is not installed or not properly configured.")


OpenCV version: 4.7.0
OpenCV is properly installed.


In [1]:
import numpy as np
import cv2

In [2]:
image_path = 'img.jpg'

- The prototxt file : defines the architecture of the model ( the different layers, the output, input, pooling...).
- The caffemodel file :  contains the actual numerical values (weights: The parameters that are learned during the training process) associated with each connection in the neural network

In [3]:
prototxt_path = 'C:/Users/malek/Desktop/stage2023/object detection/models/MobileNetSSD_deploy.prototxt'
model_path = 'C:/Users/malek/Desktop/stage2023/object detection/models/MobileNetSSD_deploy.caffemodel'

- Define the minimum confidence of the prediction

In [4]:
min_confidence = 0.05

- Define the list of different objects 

In [5]:
CLASSES = ('background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'truck',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor')

- Define the color of the rectangles of the detected objects

In [6]:
np.random.seed(543210) #for color problem solving
colors = np.random.uniform(0, 255, size=(len(CLASSES), 3)) #problem: we can have similar colors

- Load the pretrained model

In [7]:
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

In [8]:
if net.empty():
    print("Error loading the model.")
    

# 1- Image

- import the image into the neural network and resize it 

In [9]:
image = cv2.imread(image_path)
height, width = image.shape[0], image.shape[1]
blob  = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007, (300,300), 130)
#put the image into the neural network
net.setInput(blob)
detected_objects = net.forward()
print(detected_objects[0][0][0]) #result : values of the first object detected 

[ 0.         15.          0.94687384  0.50484574  0.3137808   0.6396011
  0.5986711 ]


 - 9: the type of the object which is mentionned in the list (object number 9 ) 
 - 0.975 : the confidence 
 - others : are the coordinates x and y

In [10]:
# Initialize a dictionary to count each class of object
object_counts = {class_name: 0 for class_name in CLASSES[1:]}  # Exclude 'background'

for i in range(detected_objects.shape[2]):
    confidence = detected_objects[0][0][i][2] #to get the confidence
    if confidence > min_confidence: #to test if the confidence > min then we draw a rectangle on the object
        class_id = int(detected_objects[0, 0, i, 1])
        class_name = CLASSES[class_id]
        if class_name in object_counts:
            object_counts[class_name] += 1
    #to get the exact coordinates of the whole object
        upper_left_x = int(detected_objects[0, 0, i, 3]* width)
        upper_left_y = int(detected_objects[0, 0, i, 4]* height)
        lower_right_x = int(detected_objects[0, 0, i, 5]* width)
        lower_right_y = int(detected_objects[0, 0, i, 6]* height) 
    #get the name of the object and the value of the confidence
        prediction_txt = f"{CLASSES[class_id]}: {confidence:.2f}%"  
    #draw the rectangle
        cv2.rectangle(image, (upper_left_x, upper_left_y), (lower_right_x, lower_right_y), colors[class_id], 3)
    #the position of the text ( if there is no space we change the position)
        cv2.putText(image, prediction_txt, (upper_left_x, upper_left_y - 15 if upper_left_y > 30 else upper_left_y + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, colors[class_id], 2)
                                 
        

In [11]:
# Display object counts on the image
y_position = 30
for class_name, count in object_counts.items():
    cv2.putText(image, f"{class_name}: {count}", (10, y_position), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    y_position += 30
cv2.imshow("Detected objects", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

- we use an 'object_counts' dictionary to track the number of occurrences of each object class detected in the image.

In [13]:
# count the number of objects detected
print("Number of detected objects:", len(object_counts))
for class_name, count in object_counts.items():
    print(f"{class_name}: {count} objects")

Number of detected objects: 20
aeroplane: 0 objects
bicycle: 0 objects
bird: 0 objects
boat: 0 objects
bottle: 0 objects
bus: 0 objects
car: 0 objects
cat: 0 objects
chair: 0 objects
cow: 0 objects
diningtable: 0 objects
dog: 1 objects
truck: 0 objects
motorbike: 0 objects
person: 7 objects
pottedplant: 0 objects
sheep: 0 objects
sofa: 1 objects
train: 0 objects
tvmonitor: 0 objects


# 2- video

For a video, we have to capture the frames and process each frame individually.

In [15]:
#the video path
video_path = 'traffic.mp4'  

# Open the video file
cap = cv2.VideoCapture(video_path)

# Initialize a dictionary to count each class of object
object_counts = {class_name: 0 for class_name in CLASSES[1:]}  # Exclude 'background'

while True:
    ret, frame = cap.read()  # Read a frame from the video

    if not ret:
        break  # Break the loop if the video has ended

    height, width = frame.shape[0], frame.shape[1]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007, (300, 300), 130)
    net.setInput(blob)
    detected_objects = net.forward()

    for i in range(detected_objects.shape[2]):
        confidence = detected_objects[0][0][i][2]
        if confidence > min_confidence:
            class_id = int(detected_objects[0, 0, i, 1])
            #
            class_name = CLASSES[class_id]
            if class_name in object_counts:
                object_counts[class_name] += 1
            upper_left_x = int(detected_objects[0, 0, i, 3] * width)
            upper_left_y = int(detected_objects[0, 0, i, 4] * height)
            lower_right_x = int(detected_objects[0, 0, i, 5] * width)
            lower_right_y = int(detected_objects[0, 0, i, 6] * height)
            prediction_txt = f"{CLASSES[class_id]}: {confidence:.2f}%"
            cv2.rectangle(frame, (upper_left_x, upper_left_y), (lower_right_x, lower_right_y), colors[class_id], 3)
            cv2.putText(frame, prediction_txt, (upper_left_x, upper_left_y - 15 if upper_left_y > 30 else upper_left_y + 15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, colors[class_id], 2)

    # Display object counts on the frame
    y_position = 30
    for class_name, count in object_counts.items():
        cv2.putText(frame, f"{class_name}: {count}", (10, y_position), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        y_position += 30
    
    cv2.imshow("Detected objects", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  #q : if we press q the window will be closed
        break

cap.release()
cv2.destroyAllWindows()

# 3- Webcam

In [ ]:
capp = cv2.VideoCapture(0)

while True:
    _, image = capp.read()  # to read the cam data
    height, width = image.shape[0], image.shape[1]
    blob  = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007, (300,300), 130)
    #put the image into the neural network
    net.setInput(blob)
    detected_objects = net.forward()
    print(detected_objects[0][0][0]) #result : values of the first object detected 
    # Initialize a dictionary to count each class of object
    object_counts = {class_name: 0 for class_name in CLASSES[1:]}  # Exclude 'background'

    for i in range(detected_objects.shape[2]):
        confidence = detected_objects[0][0][i][2] #to get the confidence
        if confidence > min_confidence: #to test if the confidence > min then we draw a rectangle on the object
            class_id = int(detected_objects[0, 0, i, 1])
            class_name = CLASSES[class_id]
            if class_name in object_counts:
                object_counts[class_name] += 1
        #to get the exact coordinates of the whole object
            upper_left_x = int(detected_objects[0, 0, i, 3]* width)
            upper_left_y = int(detected_objects[0, 0, i, 4]* height)
            lower_right_x = int(detected_objects[0, 0, i, 5]* width)
            lower_right_y = int(detected_objects[0, 0, i, 6]* height) 
        #get the name of the object and the value of the confidence
            prediction_txt = f"{CLASSES[class_id]}: {confidence:.2f}%"  
        #draw the rectangle
            cv2.rectangle(image, (upper_left_x, upper_left_y), (lower_right_x, lower_right_y), colors[class_id], 3)
        #the position of the text ( if there is no space we change the position)
            cv2.putText(image, prediction_txt, (upper_left_x, upper_left_y - 15 if upper_left_y > 30 else upper_left_y + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, colors[class_id], 2)
        # Display object counts on the image
    y_position = 30
    for class_name, count in object_counts.items():
        cv2.putText(image, f"{class_name}: {count}", (10, y_position), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        y_position += 30
    cv2.imshow("Detected objects", image)
    cv2.waitKey(5) #0 will freeze the image
cv2.destroyAllWindows()
cap.release()        

[ 0.         15.          0.9944688   0.23796505  0.3475642   0.9562575
  0.99722326]
[ 0.         15.          0.99390805  0.2375361   0.34672493  0.95523036
  0.9981149 ]
[ 0.         15.          0.99536383  0.23569623  0.34369165  0.9422592
  0.99555284]
[ 0.         15.          0.9951297   0.23374093  0.34657508  0.9481566
  0.99611145]
[ 0.         15.          0.9942689   0.24079299  0.3442336   0.95289063
  0.996585  ]
[ 0.         15.          0.9938076   0.24097666  0.3428484   0.94794
  0.99558437]
[ 0.         15.          0.9944008   0.2365843   0.34291318  0.9526584
  0.99500513]
[ 0.        15.         0.9943651  0.2338745  0.3451199  0.955337
  0.9949041]
[ 0.         15.          0.9955537   0.23957023  0.34187105  0.9428214
  0.9953511 ]
[ 0.         15.          0.99585634  0.23227996  0.34491163  0.94946283
  0.9944938 ]
[ 0.         15.          0.99578226  0.23386273  0.34101275  0.9440218
  0.9945922 ]
[ 0.         15.          0.9964929   0.23211113  0.343974  

[ 0.         15.          0.99671006  0.4835491   0.32333246  1.001799
  0.9974263 ]
[0.0000000e+00 1.5000000e+01 9.9868304e-01 8.2941502e-03 4.0272135e-01
 4.2557245e-01 9.9962956e-01]
[0.0000000e+00 1.5000000e+01 9.9515510e-01 1.2789518e-02 4.0547639e-01
 4.4691822e-01 9.9621314e-01]
[0.0000000e+00 1.5000000e+01 9.9556917e-01 1.2625322e-02 4.0609396e-01
 4.5342231e-01 9.9797225e-01]
[0.0000000e+00 1.5000000e+01 9.9127018e-01 1.2116760e-02 4.0389267e-01
 4.5653790e-01 9.9756968e-01]
[0.0000000e+00 1.5000000e+01 9.9011070e-01 1.4668077e-02 4.0129423e-01
 4.5710140e-01 9.9691963e-01]
[ 0.         15.          0.9902232   0.01708235  0.40376186  0.45745718
  0.9969225 ]
[ 0.         15.          0.9914932   0.02134971  0.40055335  0.46020997
  0.9976121 ]
[ 0.         15.          0.99418336  0.02035131  0.40376827  0.45762575
  0.9953164 ]
[ 0.         15.          0.9910178   0.0181344   0.4065389   0.45957285
  0.99494785]
[ 0.         15.          0.99285877  0.02075869  0.4094208   

[0.0000000e+00 1.5000000e+01 9.9516261e-01 8.6981058e-04 3.5271993e-01
 5.7383460e-01 1.0018082e+00]
[0.0000000e+00 1.5000000e+01 9.9350488e-01 1.1038780e-03 3.5209772e-01
 5.8576173e-01 1.0003327e+00]
[0.0000000e+00 1.5000000e+01 9.9334681e-01 1.9672215e-03 3.5203984e-01
 5.9558392e-01 9.9987912e-01]
[ 0.0000000e+00  1.5000000e+01  9.9072194e-01 -1.1461973e-04
  3.4275854e-01  5.8615553e-01  9.9963796e-01]
[ 0.0000000e+00  1.5000000e+01  9.9152654e-01 -4.6297908e-04
  3.4097508e-01  5.9331417e-01  9.9917674e-01]
[ 0.0000000e+00  1.5000000e+01  9.8742712e-01 -1.9259155e-03
  3.4311560e-01  5.8609319e-01  9.9928808e-01]
[0.0000000e+00 1.5000000e+01 9.8885977e-01 1.7013252e-03 3.4180495e-01
 6.0362697e-01 9.9867713e-01]
[0.0000000e+00 1.5000000e+01 9.8293787e-01 2.9098094e-03 3.4184745e-01
 6.0553658e-01 9.9934494e-01]
[0.0000000e+00 1.5000000e+01 9.8382550e-01 1.4665723e-03 3.4199765e-01
 5.9590715e-01 9.9904537e-01]
[0.0000000e+00 1.5000000e+01 9.8636091e-01 2.8520823e-05 3.4318778e-01

[ 0.         15.          0.99038637  0.22877179  0.34075868  0.7109749
  1.0062597 ]
[ 0.         15.          0.9925614   0.22088568  0.34366882  0.6987917
  0.9940882 ]
[ 0.         15.          0.99253285  0.21743746  0.3500699   0.6989175
  0.9912504 ]
[ 0.         15.          0.99052423  0.22576775  0.35079652  0.70256937
  0.985809  ]
[ 0.         15.          0.9950781   0.23009886  0.35308546  0.72022027
  0.97845167]
[ 0.         15.          0.99316436  0.23652984  0.35067162  0.71980035
  0.98060024]
[ 0.         15.          0.99626344  0.23921725  0.35028964  0.72158635
  0.97601885]
[ 0.         15.          0.99396294  0.2464527   0.34423432  0.72016555
  0.98358905]
[ 0.         15.          0.9950563   0.24543323  0.34856325  0.71965295
  0.98621017]
[ 0.         15.          0.99445     0.24314643  0.35660473  0.7215573
  0.9843198 ]
[ 0.         15.          0.99623066  0.2310958   0.36098173  0.7214992
  0.988549  ]
[ 0.         15.          0.9973667   0.24329722

[ 0.         15.          0.9542266   0.0485273   0.373299    0.7577144
  0.99930537]
[ 0.         15.          0.96906495  0.04969692  0.37337908  0.7593817
  0.9989939 ]
[ 0.         15.          0.9531139   0.04846901  0.36986476  0.7549838
  0.99802667]
[ 0.         15.          0.9736275   0.0466013   0.36996228  0.74978244
  0.9968578 ]
[ 0.         15.          0.9944635   0.05931854  0.38112855  0.7262054
  0.99154997]
[ 0.         15.          0.99785006  0.07412657  0.37562752  0.7151321
  0.998301  ]
[ 0.         15.          0.99669456  0.09075627  0.38070828  0.7134944
  0.9929573 ]
[ 0.         15.          0.9965565   0.08931917  0.38114062  0.7070612
  0.99474573]
[ 0.         15.          0.9961659   0.08502045  0.37711522  0.7053652
  0.9955667 ]
[ 0.         15.          0.995814    0.07727689  0.3749405   0.7006368
  0.99628186]
[ 0.         15.          0.9921004   0.07424381  0.37127346  0.71162045
  0.99737555]
[ 0.         15.          0.9894343   0.07136416  0.

...

[ 0.         15.          0.9892741   0.15283799  0.4633972   0.85054636
  0.99507695]
[ 0.         15.          0.9908982   0.15253747  0.46570337  0.8733461
  0.99826956]
[ 0.         15.          0.99106455  0.14823356  0.4702463   0.8995191
  1.0012658 ]
[ 0.         15.          0.99078393  0.14810061  0.47211775  0.91606283
  1.0030913 ]
[ 0.         15.          0.9913947   0.14913785  0.4705869   0.9119476
  1.0023509 ]
[ 0.         15.          0.9913947   0.14913785  0.4705869   0.9119476
  1.0023509 ]
[ 0.         15.          0.99246407  0.14715347  0.47066772  0.9378382
  1.0038856 ]
[ 0.         15.          0.99372226  0.15210783  0.4712691   0.9166169
  1.00225   ]
[ 0.         15.          0.989922    0.14774081  0.4692713   0.9056915
  1.0009125 ]
[ 0.         15.          0.9880112   0.1540376   0.46837035  0.90154433
  0.9999453 ]
[ 0.         15.          0.98794705  0.14615443  0.47088474  0.92622125
  1.0028784 ]
[ 0.         15.          0.98636395  0.1450668   

[ 0.         15.          0.98122233  0.15852436  0.41941488  0.9550209
  1.001618  ]
[ 0.         15.          0.99019974  0.15813905  0.44616064  0.9414528
  1.0000736 ]
[ 0.         15.          0.99106044  0.15484145  0.47201717  0.944263
  0.9989644 ]
[ 0.         15.          0.99174666  0.15696946  0.47624734  0.9425447
  0.99946404]
[ 0.         15.          0.9926104   0.15874869  0.4738267   0.9468444
  1.001173  ]
[ 0.         15.          0.9919883   0.1542623   0.47107887  0.9460236
  1.0008787 ]
[ 0.         15.          0.99286807  0.15806225  0.47033232  0.94162285
  1.0020628 ]
[ 0.         15.          0.99382836  0.1516389   0.46929428  0.957603
  1.0018356 ]
[ 0.         15.          0.9933662   0.16077062  0.46560544  0.9590299
  1.0021322 ]
[ 0.         15.          0.9309404   0.15700513  0.47138315  0.9653372
  1.0008557 ]
[ 0.         15.          0.9286584   0.15314928  0.47131762  0.9654317
  1.0007104 ]
[ 0.         15.          0.8392693   0.17301586  0.470

[ 0.         15.          0.9833446   0.153783    0.46594587  0.9495225
  1.0023934 ]
[ 0.         15.          0.98128694  0.14747617  0.46606764  0.94765663
  1.001757  ]
[ 0.         15.          0.9844095   0.14921463  0.46735856  0.94800293
  1.0015856 ]
[ 0.         15.          0.9799498   0.14837018  0.46686295  0.9511405
  1.0018572 ]
[ 0.         15.          0.98399127  0.15101382  0.4668601   0.94665647
  1.0010486 ]
[ 0.         15.          0.98399127  0.15101382  0.4668601   0.94665647
  1.0010486 ]
[ 0.         15.          0.98354304  0.15183824  0.465778    0.9469115
  1.0011742 ]
[ 0.         15.          0.9835781   0.15235892  0.4667764   0.9473636
  1.0021303 ]
[ 0.         15.          0.98621535  0.14894554  0.4695452   0.9492947
  1.002423  ]
[ 0.         15.          0.98638815  0.15317714  0.46837762  0.94486463
  1.0022035 ]
[ 0.         15.          0.9860205   0.15361023  0.4677867   0.95114744
  1.00255   ]
[ 0.         15.          0.9860205   0.15361023

[ 0.         15.          0.9913591   0.17216682  0.4832664   0.941962
  1.0101914 ]
[ 0.         15.          0.99523157  0.18411353  0.48578194  0.915076
  1.0109813 ]
[ 0.         15.          0.9939156   0.18138286  0.48568702  0.91676104
  1.0106825 ]
[ 0.         15.          0.99291354  0.18087381  0.48728013  0.92368907
  1.0109835 ]
[ 0.         15.          0.9941316   0.18116796  0.48736888  0.9178139
  1.0110185 ]
[ 0.         15.          0.9921799   0.16898304  0.4670209   0.88324445
  1.002946  ]
[ 0.         15.          0.9925764   0.1621212   0.46936673  0.8869108
  1.0033066 ]
[ 0.         15.          0.99419576  0.17692557  0.48389745  0.91010225
  1.0100955 ]
[ 0.         15.          0.9909117   0.17411965  0.4845085   0.9183174
  1.0096724 ]
[ 0.         15.          0.9923773   0.17476904  0.48363456  0.921381
  1.0085913 ]
[ 0.         15.          0.9911116   0.17462242  0.48522824  0.9284146
  1.0093396 ]
[ 0.         15.          0.99184775  0.16363817  0.4

[ 0.         15.          0.99246216  0.16413844  0.47786796  0.9113674
  1.0015588 ]
[ 0.         15.          0.98853314  0.16734889  0.47663957  0.91176355
  0.99940866]
[ 0.         15.          0.95842916  0.16685712  0.46095854  0.9266596
  0.9896005 ]
[ 0.         15.          0.9648608   0.1460152   0.46600273  0.9441986
  0.9934429 ]
[ 0.         15.          0.85449314  0.18749616  0.4676442   0.9417598
  1.0010215 ]
[ 0.         15.          0.37265828  0.14903677  0.45242158  0.9348078
  1.0028464 ]
[ 0.         15.          0.45498165  0.11385906  0.45594096  0.95497584
  1.0017182 ]
[ 0.         15.          0.944482    0.12335908  0.46715325  0.9582436
  1.0033629 ]
[ 0.         15.          0.98426443  0.15165493  0.4508377   0.88929856
  0.99687994]
[ 0.         15.          0.9838097   0.14868712  0.45202804  0.9209535
  0.9968996 ]
[ 0.         15.          0.98704547  0.1542413   0.45641068  0.9218079
  0.9980391 ]
[ 0.         15.          0.98466635  0.15813035  0